# Image to Information
#### Extracting Named Entities using Conditional Random Fields


# Dataset
We will be using a publicly available dataset consisting of receipts. Open a file in the below folders to look at the structure of the OCR outputs and annotations for the dataset we're using for this exercise.

In [1]:
allOCRSet = "datasets/CleanOCR" # A folder with all OCR files
annotationSet = "datasets/Annotations" # A folder with all Annotation files

# Introduction to OCR
Use Pytesseract to apply OCR on a sample receipt image.

In [6]:
import pytesseract
from PIL import Image

image_path = "datasets/SplitYB.png" # Receipt image from a restaurant

def getOCRText(image_path):
    image_data = Image.open(image_path)
    image_text = pytesseract.image_to_string(image_data)
    return image_text

print(getOCRText(image_path))

 

Split Yerba Buena

145 4th Street
San Francisco, CA 94103
Phone 415-296-8009

10/22/2018 1:35:32 PM
Order Id: AAAALGS4ACDE
33 - +++DINE HERE+++

2 VEGGIE BURGER (@10.95) $21.90
1 SONOMA $11.95
2 Sweet Potato Herb & Parmesan FREN $11.90
_ 2 Fresh Squeezed Lemonade (@2.95) $5.90

SF Employer Mandates $2.32
Sub Total $53.97
Sales Tax $4.59
Order Total $58 .56
American Express $58 .56

Card#: RRR 1000
Authorization: 573076

--> Order Closed <--

Order online at splitbread.com

 



# Text Labeling
Tokenize, Annotate and Tag a receipt for the total amount and merchant/business name

In [18]:
import pandas as pd
import os
import json

In [19]:
nerDataset = "datasets/ner-dataset.csv"
def dataAnnotation():
	allCleanOCRFiles = os.listdir(allOCRSet) # Load all Clean OCR Files
	allIXFiles = os.listdir(annotationSet) # Load all Clean OCR Files
	outfile = open(nerDataset, 'w')
	receiptNum = 1
	outfile.write('Receipt #,Word,POS,Tag' + '\n')

	for ixfile in allIXFiles:
		filename = os.fsdecode(ixfile)
		for ocrfile in allCleanOCRFiles:
			ocrFilename = os.fsdecode(ocrfile)
			if filename == ocrFilename:
				ixData = {}
				ocrCleanLine = ""
				with open(annotationSet + '/' + filename) as f1:
					ixData = json.load(f1)
				with open(allOCRSet + '/' + filename) as f2:
					ocrCleanLine = f2.readlines()
				annotationLine = ""
				wordNum = 0
				words = ocrCleanLine[0].replace(',', '').split(' ')
				isAmount = False
				isDate = False
				isMerchant = False
				beginMerchant = False
				beginDate = False

				merchantParts = ixData['company'].split(' ')
				dateParts = ixData['date'].split(' ')
				
				for word in words:
					annotationLine = ""
					word = word.replace(',', ' ')
					if wordNum == 0:
						if word == dateParts[0] and not isDate:
							annotationLine = 'Receipt: ' + str(receiptNum) + ',' + word + ',' + "DT" + ',' + 'B-date'
							isDate = True
							# Only expect a I-date if there are multiple parts to the date
							if len(dateParts) > 1:
								beginDate = True
							else:
								beginDate = False
						elif word == ixData['total'] and not isAmount:
							annotationLine = 'Receipt: ' + str(receiptNum) + ',' + word + ',' + "AM" + ',' + 'B-amt'
							isAmount = True
						elif word == merchantParts[0] and not isMerchant:
							annotationLine = 'Receipt: ' + str(receiptNum) + ',' + word + ',' + "MR" + ',' + 'B-mer'
							isMerchant = True
							beginMerchant = True
						else:
							annotationLine = 'Receipt: ' + str(receiptNum) + ',' + word + ',' + "NN" + ',' + 'O'
					else:                        
						# Amount tagging
						if word == ixData['total'] and not isAmount:
							annotationLine = ',' + word + ',' + "AM" + ',' + 'B-amt'
							isAmount = True

						# Merchant tagging
						elif word == merchantParts[0] and not isMerchant:
							beginMerchant = True
							isMerchant = True
							annotationLine = ',' + word + ',' + "MR" + ',' + 'B-mer'
						elif word in ixData['company'] and word != merchantParts[0] and word != merchantParts[-1] and beginMerchant:
							annotationLine = ',' + word + ',' + "MR" + ',' + 'I-mer'
						elif word == merchantParts[-1] and beginMerchant:
							beginMerchant = False
							annotationLine = ',' + word + ',' + "MR" + ',' + 'I-mer'

						#Other tagging
						else:
							annotationLine = ',' + word + ',' + "NN" + ',' + 'O'
					wordNum += 1						
					#print(annotationLine)
					outfile.write(annotationLine + '\n')
				receiptNum += 1
	outfile.close
    
dataAnnotation()

# NER Model Training
Train a model using Conditional Random Fields algorithm with Named Entity Recognition concept to extract amount and merchant fields.

In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn_crfsuite import CRF
from sklearn_crfsuite.metrics import flat_f1_score
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.datasets import make_blobs

In [24]:
#Reading the csv file
df = pd.read_csv('datasets/ner-dataset.csv', encoding = "ISO-8859-1")

#Display first 10 rows
df.head(10)
df.describe()
#Displaying the unique Tags
df['Tag'].unique()
#Checking null values, if any.
df.isnull().sum()
df = df.fillna(method = 'ffill')

In [25]:
class sentence(object):
    def __init__(self, df):
        self.n_sent = 1
        self.df = df
        self.empty = False
        agg = lambda s : [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(),
                                                       s['POS'].values.tolist(),
                                                       s['Tag'].values.tolist())]
        self.grouped = self.df.groupby("Receipt #").apply(agg)
        self.sentences = [s for s in self.grouped]
        
    def get_text(self):
        try:
            s = self.grouped['Receipt: {}'.format(self.n_sent)]
            self.n_sent +=1
            return s
        except:
            return None
        
#Displaying one full sentence
getter = sentence(df)
sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
sentences[0]

#sentence with its pos and tag.
sent = getter.get_text()
# print(sent)
sentences = getter.sentences

## Feature Functions

In [26]:
def listToString(s):  
    
    # initialize an empty string 
    str1 = ""  
    
    # traverse in the string   
    for ele in s:  
        str1 += ele   
    
    # return string   
    return str1

def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

## Train

In [27]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.6)

crf = CRF(algorithm = 'lbfgs',
         c1 = 0.1,
         c2 = 0.1,
         max_iterations = 100,
         all_possible_transitions = False)
crf.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CRF(algorithm='lbfgs', all_possible_transitions=False, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

## Test 

In [28]:
#Predicting on the test set.
y_pred = crf.predict(X_test)

for i in range(len(y_pred)):
	prediction = y_pred[i]
	testList = X_test[i]
	testSentence = ""
	for testTuple in testList:
		testSentence = testSentence + testTuple['word.lower()'] + ' '
	words = testSentence.split(" ")
	x = 0
	for wordPrediction in prediction:
		if wordPrediction == 'B-date' or wordPrediction == 'B-amt' or wordPrediction == 'B-mer' or wordPrediction == 'I-mer' or wordPrediction == 'I-date':
			print(words[x],wordPrediction)
		x +=1

grand B-mer
lux I-mer


## Generate Report
Print the results of how your model was trained and tested to verify

In [29]:
f1_score = flat_f1_score(y_test, y_pred, average = 'weighted')
print(f1_score)

report = flat_classification_report(y_test, y_pred)
print(report)

0.9639573493197691
              precision    recall  f1-score   support

       B-amt       0.00      0.00      0.00         2
       B-mer       1.00      0.50      0.67         2
       I-mer       1.00      0.33      0.50         3
           O       0.97      1.00      0.99       177

    accuracy                           0.97       184
   macro avg       0.74      0.46      0.54       184
weighted avg       0.96      0.97      0.96       184



/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass labels=None as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Practical Application
Go back above to Text Labeling section and add the below code under #Date Tagging section. Continue with the steps to generate a model that now extracts 3 fields.

In [ ]:
						# Date tagging
						if word == dateParts[0] and not isDate:
							annotationLine = ',' + word + ',' + "DT" + ',' + 'B-date'
							isDate = True
							# Only expect a I-date if there are multiple parts to the date
							if len(dateParts) > 1:
								# print("This date has many parts: " + ixData['date'])
								beginDate = True
							else:
								beginDate = False
						elif word in ixData['date'] and word != dateParts[0] and word != dateParts[-1] and beginDate:
							annotationLine = ',' + word + ',' + "DT" + ',' + 'I-date'
						elif word == dateParts[-1] and beginDate:
							beginDate = False
							annotationLine = ',' + word + ',' + "DT" + ',' + 'I-date'

## Test End to End by uploading a new image - OCR to NER

In [37]:
new_image_path = "datasets/lunch.png"
test_file = getOCRText(new_image_path)
print(test_file)

Bon Appetit
Oracle Redwood
Cafe 300

ES
rm CHK 2042
6/14/2018 | 1.05 PM
1! & g Salad, Large Ceasar 5 09
1 Side, Chicken 2.39

 

eeeERKHK( 03

VISA:087861

Credit Authorization 6,13
VISA $8.13
EKKKKKKKKKKD 4D

Subtota | $7.48
Tax $0.65
Payment $8.13

Change Due $O.00

Seeerterene Check Closed -----------
6/14/2018 1:05 PM

   



In [38]:
def get_test_sentence(test_file):
    #Reading the csv file
    trial = pd.read_csv(test_file, encoding = "ISO-8859-1")

    #Display first 10 rows
    trial.head(10)
    trial.describe()
    #Displaying the unique Tags
    trial['Tag'].unique()
    #Checking null values, if any.
    trial.isnull().sum()
    trial = trial.fillna(method = 'ffill')
    trial_getter = sentence(trial) # Check last token - seems to be repeating 
    trial_sentences = [" ".join([s[0] for s in sent]) for sent in trial_getter.sentences]
    trial_sentences[0]

    #sentence with its pos and tag.
    trial_sent = trial_getter.get_text()
    trial_sentences = trial_getter.sentences
    return trial_sentences

trial_sentences = get_test_sentence(test_file)

FileNotFoundError: [Errno 2] No such file or directory: 'Bon Appetit\nOracle Redwood\nCafe 300\n\nES\nrm CHK 2042\n6/14/2018 | 1.05 PM\n1! & g Salad, Large Ceasar 5 09\n1 Side, Chicken 2.39\n\n \n\neeeERKHK( 03\n\nVISA:087861\n\nCredit Authorization 6,13\nVISA $8.13\nEKKKKKKKKKKD 4D\n\nSubtota | $7.48\nTax $0.65\nPayment $8.13\n\nChange Due $O.00\n\nSeeerterene Check Closed -----------\n6/14/2018 1:05 PM\n\n   \n\x0c'

In [ ]:
tX = [sent2features(s) for s in trial_sentences]
ty = [sent2labels(s) for s in trial_sentences]

#Predicting on the one test set.
X_test = tX
y_test = ty
y_pred = crf.predict(X_test)

f1_score = flat_f1_score(y_test, y_pred, average = 'weighted')
print(f1_score)

report = flat_classification_report(y_test, y_pred)
print(report)